# Chromassembly: DNA Coloring Algorithm

Contributors: Dr. Jason Kahn (Brookhaven), Sarah Hong (Columbia)

This notebook shows the final workflow for how to create a lattice structure and paint it.

In [ ]:
%load_ext autoreload
%autoreload 2

## Create the lattice
Run the below cell to open a GUI which allows you to quickly create arbitrary lattice structures. These lattices are encoded as numpy arrays and are saved to a (renamable) file called `lattice.npy` in the `notebooks/data` subdirectory. You can also use this feature to save and reload arbitrary lattice designs you've previously created.

In [ ]:
%gui qt

import sys
sys.path.append('../')

import numpy as np
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication
from app.design.Designer import RunDesigner

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()
    
    designer = RunDesigner(app)
    input_lattice = designer.run()
    
    if input_lattice is not None:
        print('Lattice received.')
        # rename this to whatever you want to call your lattice
        np.save("data/lattice.npy", input_lattice) 
    else:
        print("No lattice received.")

Saved lattice:
[[[1 1 1]
  [0 0 0]
  [1 1 1]]

 [[1 0 1]
  [0 1 0]
  [1 0 1]]

 [[1 1 1]
  [0 0 0]
  [1 1 1]]]

Lattice received.


## Running the algorithm

### Load lattice from file
We can load in the lattice we created with `np.load(filename.npy)`. Since the lattice is just a numpy array, you can also create your own arbitrary lattices without the designer by creating arbitrary numpy arrays yourself.

Some examples to run:
- `double_oreo.npy`
- `perovskite.npy`

### Painting algorithm
All you need to do to run the algorithm is to create a Lattice instance, `compute_symmetries()`, create a Painter instance, and `paint_lattice()`.

The example workflow is detailed below.

In [ ]:
from algorithm.lattice.Lattice import Lattice
from algorithm.painting.Painter import Painter

input_lattice = np.load('data/lattice.npy')

# An example lattice structure you can load in
# input_lattice = np.load('data/double_oreo.npy')

# Initialize the data structures and compute all symmetries
lattice = Lattice(input_lattice)
lattice.compute_symmetries()

# Paint the lattice
painter = Painter(lattice, verbose=False) # Set verbose=True to print debug
painter.paint_lattice()

## Optional: Binding Flexibility
In this section, we introduce an optional "binding flexibility" parameter, which modifies the final painting result to be stricter or looser. This is motivated by the observation that the geometry / symmetries of specific structures may perform better under different constraints, and we offer the user the option to make it more/less strict.

### Background
**Binding flexibility 1:** Reduces specificity (less colors)
- BF 1 turns all bonds to voxels in the same automorphism equivalence class are repainted to be the same color. 
- In other words, we repaint all bonds between all voxels with some symmetry between them to be the same color.

**Binding flexibility 2:** No change
- BF 2 is the default result from the Painter.paint() algorithm. It generally leads to the best results, so most systems are painted with this.

**Binding flexibility 3:** Adds specificity (more colors)
- Introducing a maximum cutoff ratio (= structural bonds / total bonds) per voxel.
- For all voxels in the lattice with a cutoff ratio higher than the maximum, we repaint a NEW color ontop of a structural bond on that voxel to lower the ratio to satisfy the constraint.
- This ratio is motivated by the observation that having more structural bonds on the same voxel makes it more likely to erroneously bind in ways we don't want.

### Usage
To repaint the lattice with a specific binding flexibility, create a new `BindingFlexibility` instance specified with a given (painted) lattice object. Then run either `binding_flexibility1()` or `binding_flexibility3()` and assign it to a new variable to create a NEW lattice instance which can be visualized in the app. An example usage can be run below.

In [ ]:
from algorithm.painting.BindingFlexibility import BindingFlexibility

bf = BindingFlexibility(lattice)

bf1_lattice = bf.binding_flexibility_1()
bf3_lattice = bf.binding_flexibility_3(max_cutoff_ratio=1/6)

## Visualize the painted lattice

Run the below cell to view the painted lattice as a result from the algorithm.

To view a specific binding flexibility in the app, just supply the given binding flexibility lattice as the argument to `RunVisualizer`.

In [ ]:
%gui qt
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication

from app.visualize.Visualizer import RunVisualizer

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    visualizeWindow = RunVisualizer(lattice, app) # insert bf_lattice here if you want

## Minimal origami set

You can run `lattice.final_df()` to get an interpretable, final dataframe which contains all voxel/bond information.

In [17]:
final_df = lattice.final_df()
final_df

Voxel                      Bond Colors               
     ID Material Coordinates          +x -x +y -y +z -z
0     0        1   (0, 1, 1)           1  1  1  1  1  1
1     1        0   (1, 1, 1)          -1 -1  2  2  2  2
2     2        0   (0, 0, 1)           2  2 -1 -1  2  2
3     3        2   (1, 0, 1)          -2 -2 -2 -2  3  3
4     4        0   (0, 1, 0)           2  2  2  2 -1 -1
5     5        2   (1, 1, 0)          -2 -2  3  3 -2 -2
6     6        2   (0, 0, 0)           3  3 -2 -2 -2 -2
7     7        3   (1, 0, 0)          -3 -3 -3 -3 -3 -3